In [1]:
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

df = sqlContext.read.load('file:///home/cloudera/Downloads/big-data/daily_weather.csv', 
                          format='com.databricks.spark.csv', 
                          header='true',inferSchema='true')

In [2]:
df.describe().toPandas().transpose()

0                    1                   2  \
summary                 count                 mean              stddev   
number                   1095                547.0  316.24357700987383   
air_pressure_9am         1092    918.8825513138094   3.184161180386833   
air_temp_9am             1090    64.93300141287072  11.175514003175877   
avg_wind_direction_9am   1091    142.2355107005759   69.13785928889189   
avg_wind_speed_9am       1092     5.50828424225493  4.5528134655317185   
max_wind_direction_9am   1092   148.95351796516923   67.23801294602953   
max_wind_speed_9am       1091    7.019513529175272   5.598209170780958   
rain_accumulation_9am    1089  0.20307895225211126  1.5939521253574893   
rain_duration_9am        1092    294.1080522756142  1598.0787786601481   
relative_humidity_9am    1095    34.24140205923536  25.472066802250055   
relative_humidity_3pm    1095    35.34472714825898  22.524079453587273   

                                         3                   4  
summary                                min                 max  
number                                   0                1094  
air_pressure_9am         907.9900000000024   929.3200000000012  
air_temp_9am            36.752000000000685   98.90599999999992  
avg_wind_direction_9am  15.500000000000046               343.4  
avg_wind_speed_9am        0.69345139999974  23.554978199999763  
max_wind_direction_9am   28.89999999999991  312.19999999999993  
max_wind_speed_9am      1.1855782000000479   29.84077959999996  
rain_accumulation_9am                  0.0   24.01999999999907  
rain_duration_9am                      0.0             17704.0  
relative_humidity_9am    6.090000000001012    92.6200000000002  
relative_humidity_3pm   5.3000000000006855    92.2500000000003

In [3]:
df.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141287072|
| stddev|11.175514003175877|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [4]:
df.count()

1095

In [5]:
removeAllDF = df.na.drop()

In [6]:
removeAllDF.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1064|
|   mean| 65.02260949558733|
| stddev|11.168033449415704|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [7]:
removeAllDF.count()

1064

In [8]:
from pyspark.sql.functions import avg

In [9]:
imputeDF = df

In [10]:
for x in imputeDF.columns:
    meanValue = removeAllDF.agg(avg(x)).first()[0]
    print(x, meanValue)
    imputeDF = imputeDF.na.fill(meanValue, [x])

number 545.0018796992481
air_pressure_9am 918.9031798641051
air_temp_9am 65.02260949558733
avg_wind_direction_9am 142.30675564934037
avg_wind_speed_9am 5.48579305071369
max_wind_direction_9am 148.48042413321315
max_wind_speed_9am 6.999713658875691
rain_accumulation_9am 0.18202347650615522
rain_duration_9am 266.3936973996037
relative_humidity_9am 34.07743985327709
relative_humidity_3pm 35.14838093290533


In [11]:
df.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141287072|
| stddev|11.175514003175877|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [12]:
imputeDF.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1095|
|   mean| 64.93341058219818|
| stddev| 11.14994819992023|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [13]:
removeAllDF.describe(['air_pressure_9am']).show()

+-------+------------------+
|summary|  air_pressure_9am|
+-------+------------------+
|  count|              1064|
|   mean| 918.9031798641051|
| stddev|3.1790395342048496|
|    min| 907.9900000000024|
|    max| 929.3200000000012|
+-------+------------------+



In [14]:
removeAllDF.filter((removeAllDF["air_pressure_9am"] > 911.736) & (removeAllDF["air_pressure_9am"] < 914.67)).count()

77

In [15]:
from pyspark.sql.functions import min

In [16]:
removeAllDF1 = df.na.drop()

In [17]:
imputeDF1 = df

In [18]:
for x in imputeDF1.columns:
    minValue = removeAllDF1.agg(min(x)).first()[0]
    print(x, minValue)
    imputeDF1 = imputeDF1.na.fill(minValue, [x])

number 0
air_pressure_9am 907.9900000000024
air_temp_9am 36.752000000000685
avg_wind_direction_9am 15.500000000000046
avg_wind_speed_9am 0.69345139999974
max_wind_direction_9am 28.89999999999991
max_wind_speed_9am 1.1855782000000479
rain_accumulation_9am 0.0
rain_duration_9am 0.0
relative_humidity_9am 6.090000000001012
relative_humidity_3pm 5.3000000000006855


In [19]:
imputeDF1.filter(imputeDF1["air_temp_9am"] < 42.292).count()

28

In [20]:
df.describe(['air_pressure_9am']).show()

+-------+-----------------+
|summary| air_pressure_9am|
+-------+-----------------+
|  count|             1092|
|   mean|918.8825513138094|
| stddev|3.184161180386833|
|    min|907.9900000000024|
|    max|929.3200000000012|
+-------+-----------------+



1095 - 1092 = 3

In [21]:
df.describe().show()

+-------+------------------+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+------------------+---------------------+---------------------+
|summary|            number| air_pressure_9am|      air_temp_9am|avg_wind_direction_9am|avg_wind_speed_9am|max_wind_direction_9am|max_wind_speed_9am|rain_accumulation_9am| rain_duration_9am|relative_humidity_9am|relative_humidity_3pm|
+-------+------------------+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+------------------+---------------------+---------------------+
|  count|              1095|             1092|              1090|                  1091|              1092|                  1092|              1091|                 1089|              1092|                 1095|                 1095|
|   mean|             547.0|918.8825513138094| 64.9330014128

1089 has most number of missing values = rain_accumulation_9am